In [39]:
import pickle
with open('../Data/allplayerGameLogs.pickle', 'rb') as handle:
  allplayerGameLogs = pickle.load(handle)

In [2]:
import pandas as pd

In [273]:
allplayerGameLogs.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22015,203112,0021500889,"FEB 29, 2016",SAC vs. OKC,L,28,5,9,0.556,...,0,4,2,3,0,0,3,10,-16,1
1,22015,203112,0021500867,"FEB 26, 2016",SAC vs. LAC,L,15,1,1,1.000,...,0,0,0,0,1,0,3,4,-12,1
2,22015,203112,0021500854,"FEB 24, 2016",SAC vs. SAS,L,23,6,9,0.667,...,2,2,0,2,0,1,4,15,13,1
3,22015,203112,0021500842,"FEB 23, 2016",SAC @ DEN,W,13,0,2,0.000,...,3,3,0,0,3,0,1,0,-1,1
4,22015,203112,0021500816,"FEB 19, 2016",SAC vs. DEN,W,15,0,0,0.000,...,3,3,0,0,0,1,1,0,2,1


In [4]:
allplayerGameLogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18964 entries, 0 to 42
Data columns (total 27 columns):
SEASON_ID          18964 non-null object
Player_ID          18964 non-null float64
Game_ID            18964 non-null object
GAME_DATE          18964 non-null object
MATCHUP            18964 non-null object
WL                 18964 non-null object
MIN                18964 non-null float64
FGM                18964 non-null float64
FGA                18964 non-null float64
FG_PCT             18964 non-null float64
FG3M               18964 non-null float64
FG3A               18964 non-null float64
FG3_PCT            18964 non-null float64
FTM                18964 non-null float64
FTA                18964 non-null float64
FT_PCT             18964 non-null float64
OREB               18964 non-null float64
DREB               18964 non-null float64
REB                18964 non-null float64
AST                18964 non-null float64
STL                18964 non-null float64
BLK              

Modify the datetime

In [282]:
allplayerGameLogs['GAME_DATE'] = pd.to_datetime(allplayerGameLogs['GAME_DATE'])

In [283]:
allplayerGameLogs['GAME_DATE'].head()

0   2016-02-29
1   2016-02-26
2   2016-02-24
3   2016-02-23
4   2016-02-19
Name: GAME_DATE, dtype: datetime64[ns]

In [284]:
del allplayerGameLogs['VIDEO_AVAILABLE']

In [285]:
allplayerGameLogs.columns

Index([u'SEASON_ID', u'Player_ID', u'Game_ID', u'GAME_DATE', u'MATCHUP', u'WL',
       u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM',
       u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK',
       u'TOV', u'PF', u'PTS', u'PLUS_MINUS'],
      dtype='object')

Add columns to indicate whether the player has double-double or triple-double.

In [286]:
dd = ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['AST']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['REB']>=10))| \
     ((allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10))| \
     ((allplayerGameLogs['REB']>=10) & (allplayerGameLogs['BLK']>=10))| \
     ((allplayerGameLogs['STL']>=10) & (allplayerGameLogs['BLK']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['BLK']>=10))

In [287]:
sum(dd)

1517

In [288]:
allplayerGameLogs['DouBL']= dd

In [289]:
ttt = (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['BLK']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['BLK']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['AST']>=10)| \
      (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10)

In [290]:
sum(ttt)

47

In [291]:
allplayerGameLogs['TriBL']= ttt

calculate the fantasy point for a player to add a column in the data frame

In [292]:
allplayerGameLogs['FanPTs'] = 3.5 * allplayerGameLogs['FG3M'] + 1*allplayerGameLogs['FTM'] \
+ 2*(allplayerGameLogs['FGM']-allplayerGameLogs['FG3M']) \
+ 1.25*allplayerGameLogs['REB']+1.5*allplayerGameLogs['AST'] \
+ 2*allplayerGameLogs['STL']+ 2*allplayerGameLogs['BLK'] \
+ 1.5*allplayerGameLogs['DouBL'] + 3*allplayerGameLogs['TriBL'] \
- 0.5*allplayerGameLogs['TOV']

Test case to verify

In [293]:
davis = allplayerGameLogs[allplayerGameLogs['Player_ID']==203076]

In [294]:
davis[['GAME_DATE','MATCHUP','PTS','AST','REB','BLK','STL','TOV','DouBL','TriBL','FanPTs']].head()

,GAME_DATE,MATCHUP,PTS,AST,REB,BLK,STL,TOV,DouBL,TriBL,FanPTs
0,2016-02-25,NOP vs. OKC,30,2,6,4,1,2,False,False,50.50
1,2016-02-23,NOP @ WAS,9,3,20,1,2,3,False,False,43.00
2,2016-02-21,NOP @ DET,59,4,20,1,0,2,True,False,93.50
3,2016-02-19,NOP vs. PHI,34,1,7,2,2,2,False,False,51.25
4,2016-02-11,NOP @ OKC,23,2,3,0,2,1,False,False,33.25


Already checked.. it does match the official record from my playing history.

In [295]:
allplayerFantasyGameLogs = allplayerGameLogs.set_index('GAME_DATE')

In [296]:
allplayerFantasyGameLogs = allplayerFantasyGameLogs.sort_index(axis=0)

In [297]:
allplayerFantasyGameLogs.head()

,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,0,...,3,1,0,0,5,0,-21,False,False,6.50
2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,0,...,2,0,2,3,2,5,-1,False,False,16.75
2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,0,...,1,3,0,3,3,8,-8,False,False,20.25
2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,1,...,4,1,0,3,1,3,-14,False,False,10.00
2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,0,...,2,2,1,0,1,17,3,False,False,32.25


Use join to get the corresponding player names

In [309]:
import pickle
with open('../Data/playid_df.pickle', 'wb') as handle:
    pickle.dump(playid_df, handle)

In [54]:
playid_df = pd.DataFrame(playid_data)

In [57]:
playid_df['fullName'] = playid_df['firstName'] + ' ' + playid_df['lastName']

In [58]:
playid_df.head()

,firstName,lastName,playerId,teamId,fullName
0,Quincy,Acy,203112,1610612758,Quincy Acy
1,Jordan,Adams,203919,1610612763,Jordan Adams
2,Steven,Adams,203500,1610612760,Steven Adams
3,Arron,Afflalo,201167,1610612752,Arron Afflalo
4,Alexis,Ajinca,201582,1610612740,Alexis Ajinca


In [310]:
allplayerFantasyGameLogs = pd.merge(allplayerFantasyGameLogs.reset_index(), playid_df[['playerId' \
     ,'fullName']], left_on='Player_ID',right_on='playerId', how='left')

In [311]:
del allplayerFantasyGameLogs['playerId']

In [312]:
allplayerFantasyGameLogs.head()

,GAME_DATE,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName
0,2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,...,1,0,0,5,0,-21,False,False,6.50,Nate Robinson
1,2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,...,0,2,3,2,5,-1,False,False,16.75,Timofey Mozgov
2,2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,...,3,0,3,3,8,-8,False,False,20.25,Alonzo Gee
3,2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,...,1,0,3,1,3,-14,False,False,10.00,Steve Blake
4,2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,...,2,1,0,1,17,3,False,False,32.25,Jimmy Butler


Add column of player position

In [306]:
with open('../Data/players_with_positioninfo.json', 'rb') as handle:
  playsinfo = json.load(handle)

In [313]:
playsinfo = pd.DataFrame(playsinfo)

In [314]:
playsinfo = playsinfo.transpose()

In [315]:
playsinfo = playsinfo.reset_index()

In [316]:
playsinfo = playsinfo.rename(columns={'index':'fullName'})

In [317]:
pos_dict = {'Power Forward':'PF', 'Small Forward':'SF', 'Center':'C','Shooting Guard':'SG', \
            'Point Guard':'PG', 'Forward':'PF', 'CenterHeight: 6-11':'C'}

In [319]:
playsinfo['position1'] = playsinfo['positions'].apply(lambda x: pos_dict[x[0]] if (len(x)>0) else None)

In [320]:
playsinfo['position2'] = playsinfo['positions'].apply(lambda x: pos_dict[x[1]] if (len(x)>1) else None)

Data cleaning, e.g replace 'J.J. Redick' to 'JJ Redick'

In [228]:
import re

In [321]:
playsinfo['fullName'] = playsinfo['fullName'].apply(lambda x: re.sub(r'\.','',x))

In [322]:
playsinfo.head()

,fullName,gamelog_data,gamelog_url_list,height,name,overview_url,overview_url_content,positions,weight,position1,position2
0,AJ Price,None,[http://www.basketball-reference.com/players/p...,6-2,A.J. Price,http://www.basketball-reference.com/players/p/...,A.J. Price NBA Stats | Basketball-Reference.co...,"[Point Guard, Shooting Guard]",181,PG,SG
1,Aaron Brooks,None,[http://www.basketball-reference.com/players/b...,6-0,Aaron Brooks,http://www.basketball-reference.com/players/b/...,Aaron Brooks NBA Stats | Basketball-Reference....,[Point Guard],161,PG,None
2,Aaron Gordon,None,[http://www.basketball-reference.com/players/g...,6-9,Aaron Gordon,http://www.basketball-reference.com/players/g/...,Aaron Gordon NBA Stats | Basketball-Reference....,[Power Forward],220,PF,None
3,Aaron Harrison,None,[http://www.basketball-reference.com/players/h...,6-6,Aaron Harrison,http://www.basketball-reference.com/players/h/...,Aaron Harrison NBA Stats | Basketball-Referenc...,[Shooting Guard],210,SG,None
4,Adonis Thomas,None,[http://www.basketball-reference.com/players/t...,6-7,Adonis Thomas,http://www.basketball-reference.com/players/t/...,Adonis Thomas NBA Stats | Basketball-Reference...,[Small Forward],200,SF,None


In [324]:
with open('../Data/players_with_positioninfo_df.pickle', 'wb') as handle:
  pickle.dump(playsinfo,handle)

In [328]:
with open('../Data/allPlayerBios.pickle', 'rb') as handle:
  playerBios = pickle.load(handle)

In [329]:
playerBios.head()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,...,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,GAMES_PLAYED_FLAG
0,203112,Quincy,Acy,Quincy Acy,"Acy, Quincy",Q. Acy,1990-10-06T00:00:00,Baylor,USA,Baylor/USA,...,1610612758,Kings,SAC,kings,Sacramento,quincy_acy,2012,2015,Y,Y
0,203919,Jordan,Adams,Jordan Adams,"Adams, Jordan",J. Adams,1994-07-08T00:00:00,UCLA,USA,UCLA/USA,...,1610612763,Grizzlies,MEM,grizzlies,Memphis,jordan_adams,2014,2015,Y,Y
0,203500,Steven,Adams,Steven Adams,"Adams, Steven",S. Adams,1993-07-20T00:00:00,Pittsburgh,New Zealand,Pittsburgh/New Zealand,...,1610612760,Thunder,OKC,thunder,Oklahoma City,steven_adams,2013,2015,N,Y
0,201167,Arron,Afflalo,Arron Afflalo,"Afflalo, Arron",A. Afflalo,1985-10-15T00:00:00,UCLA,USA,UCLA/USA,...,1610612752,Knicks,NYK,knicks,New York,arron_afflalo,2007,2015,N,Y
0,201582,Alexis,Ajinca,Alexis Ajinca,"Ajinca, Alexis",A. Ajinca,1988-05-06T00:00:00,"Saint Etienne, France",France,"Saint Etienne, France",...,1610612740,Pelicans,NOP,pelicans,New Orleans,alexis_ajinca,2008,2015,Y,Y


In [330]:
playerBios = pd.merge(playerBios, playsinfo[['fullName','position1','position2']] \
                      ,left_on='DISPLAY_FIRST_LAST',right_on='fullName', how='left')

In [331]:
playerBios['position1'].unique() #with NA values

array(['PF', 'SG', 'C', 'SF', nan, 'PG'], dtype=object)

In [255]:
playerBios.head()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,...,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,GAMES_PLAYED_FLAG,fullName,position1,position2
0,203112,Quincy,Acy,Quincy Acy,"Acy, Quincy",Q. Acy,1990-10-06T00:00:00,Baylor,USA,Baylor/USA,...,kings,Sacramento,quincy_acy,2012,2015,Y,Y,Quincy Acy,PF,SF
1,203919,Jordan,Adams,Jordan Adams,"Adams, Jordan",J. Adams,1994-07-08T00:00:00,UCLA,USA,UCLA/USA,...,grizzlies,Memphis,jordan_adams,2014,2015,Y,Y,Jordan Adams,SG,None
2,203500,Steven,Adams,Steven Adams,"Adams, Steven",S. Adams,1993-07-20T00:00:00,Pittsburgh,New Zealand,Pittsburgh/New Zealand,...,thunder,Oklahoma City,steven_adams,2013,2015,N,Y,Steven Adams,C,None
3,201167,Arron,Afflalo,Arron Afflalo,"Afflalo, Arron",A. Afflalo,1985-10-15T00:00:00,UCLA,USA,UCLA/USA,...,knicks,New York,arron_afflalo,2007,2015,N,Y,Arron Afflalo,SG,SF
4,201582,Alexis,Ajinca,Alexis Ajinca,"Ajinca, Alexis",A. Ajinca,1988-05-06T00:00:00,"Saint Etienne, France",France,"Saint Etienne, France",...,pelicans,New Orleans,alexis_ajinca,2008,2015,Y,Y,Alexis Ajinca,C,PF


In [332]:
del playerBios['fullName']

In [333]:
playerBios[playerBios[['position1']].isnull().any(axis=1)]

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,...,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,GAMES_PLAYED_FLAG,position1,position2
19,203648,Thanasis,Antetokounmpo,Thanasis Antetokounmpo,"Antetokounmpo, Thanasis",T. Antetokounmpo,1992-07-18T00:00:00,Greece,None,Greece,...,NYK,knicks,New York,thanasis_antetokounmpo,2015,2015,Y,Y,NaN,NaN
26,201571,D.J.,Augustin,D.J. Augustin,"Augustin, D.J.",D. Augustin,1987-11-10T00:00:00,Texas,USA,Texas/USA,...,DEN,nuggets,Denver,dj_augustin,2008,2015,N,Y,NaN,NaN
31,200826,Jose Juan,Barea,Jose Juan Barea,"Barea, Jose Juan",J. Barea,1984-06-26T00:00:00,Northeastern,Puerto Rico,Northeastern/Puerto Rico,...,DAL,mavericks,Dallas,jose_barea,2006,2015,Y,Y,NaN,NaN
85,1626176,Rakeem,Christmas,Rakeem Christmas,"Christmas, Rakeem",R. Christmas,1991-12-01T00:00:00,Syracuse,USA,Syracuse/USA,...,IND,pacers,Indiana,rakeem_christmas,2015,2015,Y,Y,NaN,NaN
121,1626251,Duje,Dukan,Duje Dukan,"Dukan, Duje",D. Dukan,1991-12-04T00:00:00,Wisconsin,Croatia,Wisconsin/Croatia,...,SAC,kings,Sacramento,duje_dukan,2015,2015,Y,Y,NaN,NaN
129,203954,Joel,Embiid,Joel Embiid,"Embiid, Joel",J. Embiid,1994-03-16T00:00:00,Kansas,Cameroon,Kansas/Cameroon,...,PHI,sixers,Philadelphia,joel_embiid,2014,2015,N,N,NaN,NaN
173,203501,Tim,Hardaway Jr.,Tim Hardaway Jr.,"Hardaway Jr., Tim",T. Hardaway Jr.,1992-03-16T00:00:00,Michigan,USA,Michigan/USA,...,ATL,hawks,Atlanta,timothy_hardaway,2013,2015,Y,Y,NaN,NaN
185,101236,Charles,Hayes,Charles Hayes,"Hayes, Charles",C. Hayes,1983-06-11T00:00:00,Kentucky,USA,Kentucky/USA,...,,,,chuck_hayes,2005,2015,Y,Y,NaN,NaN
192,2403,Nene,Hilario,Nene,Nene,Nene,1982-09-13T00:00:00,"Sao Carlos, Brazil",Brazil,Vasco de Gama/Brazil,...,WAS,wizards,Washington,nene,2002,2015,N,Y,NaN,NaN
207,203962,Josh,Huestis,Josh Huestis,"Huestis, Josh",J. Huestis,1991-12-19T00:00:00,Stanford,None,Stanford/USA,...,OKC,thunder,Oklahoma City,josh_huestis,2015,2015,Y,Y,NaN,NaN


Hard code the missing information

In [334]:
tmp = playerBios[playerBios[['position1']].isnull().any(axis=1)]

In [335]:
playerBios[playerBios['DISPLAY_FIRST_LAST'].isin(tmp['DISPLAY_FIRST_LAST'])].index

Int64Index([ 19,  26,  31,  85, 121, 129, 173, 185, 192, 207, 279, 285, 291,
            300, 316, 328, 347, 371, 396, 405, 463],
           dtype='int64')

In [336]:
to_fix_index = playerBios[playerBios['DISPLAY_FIRST_LAST'].isin(tmp['DISPLAY_FIRST_LAST'])].index

In [337]:
playerBios.ix[to_fix_index,'position1'] = ['SG','PG','PG','C','PF','C','SG','C','PF','SF','SG','PF'\
                                          ,'SG','PG','PF','SF','PF','SF','SG','PF','PG']

Add position information for each player

In [338]:
allplayerFantasyGameLogs = pd.merge(allplayerFantasyGameLogs,playerBios[['PERSON_ID','position1']], left_on='Player_ID', \
                                    right_on='PERSON_ID', how='left')

In [340]:
del allplayerFantasyGameLogs['PERSON_ID']

In [341]:
allplayerFantasyGameLogs.head()

,GAME_DATE,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName,position1
0,2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,...,0,0,5,0,-21,False,False,6.50,Nate Robinson,PG
1,2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,...,2,3,2,5,-1,False,False,16.75,Timofey Mozgov,C
2,2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,...,0,3,3,8,-8,False,False,20.25,Alonzo Gee,SF
3,2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,...,0,3,1,3,-14,False,False,10.00,Steve Blake,PG
4,2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,...,1,0,1,17,3,False,False,32.25,Jimmy Butler,SG


In [342]:
with open('../Data/allplayerFantasyGameLogs.pickle', 'wb') as handle:
  pickle.dump(allplayerFantasyGameLogs,handle)

Add team information for each player

In [42]:
import pickle
with open('../Data/allplayerFantasyGameLogs.pickle', 'rb') as handle:
  allplayerFantasyGameLogs = pickle.load(handle)

In [20]:
allplayerFantasyGameLogs['Team'] = allplayerFantasyGameLogs['MATCHUP'].map(lambda x: x.split(' ')[0])

In [21]:
allplayerFantasyGameLogs['OpponentTeam'] = allplayerFantasyGameLogs['MATCHUP'].map(lambda x: x.split(' ')[2])

In [26]:
allplayerFantasyGameLogs['HomeGame'] = allplayerFantasyGameLogs['MATCHUP'].map(lambda x: 0 if x.split(' ')[1]=='@' else 1)

In [27]:
allplayerFantasyGameLogs[allplayerFantasyGameLogs['Player_ID']==202710].head()

,GAME_DATE,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName,position1,Team,OpponentTeam,HomeGame
4,2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,...,17,3,False,False,32.25,Jimmy Butler,SG,CHI,CLE,1
293,2015-10-28,22015,202710,0021500006,CHI @ BKN,W,34,9,11,0.818,...,24,23,False,False,43.00,Jimmy Butler,SG,CHI,BKN,0
566,2015-10-30,22015,202710,0021500025,CHI @ DET,L,42,5,19,0.263,...,23,-2,True,False,44.25,Jimmy Butler,SG,CHI,DET,0
949,2015-11-01,22015,202710,0021500043,CHI vs. ORL,W,33,4,10,0.400,...,14,4,False,False,38.50,Jimmy Butler,SG,CHI,ORL,1
1233,2015-11-03,22015,202710,0021500053,CHI @ CHA,L,32,10,16,0.625,...,26,-19,False,False,34.75,Jimmy Butler,SG,CHI,CHA,0


In [46]:
with open('../Data/allplayerFantasyGameLogs.pickle', 'wb') as handle:
  pickle.dump(allplayerFantasyGameLogs,handle)

Fix some missing values

In [30]:
allplayerFantasyGameLogs[allplayerFantasyGameLogs['Player_ID']==2403].head()

,GAME_DATE,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName,position1,Team,OpponentTeam,HomeGame
80,2015-10-28,22015,2403,0021500004,WAS @ ORL,W,12,0,6,0.00,...,1,-13,False,False,6.50,NaN,PF,WAS,ORL,0
687,2015-10-30,22015,2403,0021500027,WAS @ MIL,W,22,3,3,1.00,...,9,17,False,False,24.50,NaN,PF,WAS,MIL,0
794,2015-10-31,22015,2403,0021500034,WAS vs. NYK,L,18,3,4,0.75,...,8,2,False,False,14.00,NaN,PF,WAS,NYK,1
1315,2015-11-04,22015,2403,0021500061,WAS vs. SAS,W,20,3,5,0.60,...,6,-1,False,False,11.75,NaN,PF,WAS,SAS,1
1648,2015-11-06,22015,2403,0021500076,WAS @ BOS,L,15,1,4,0.25,...,5,-20,False,False,9.00,NaN,PF,WAS,BOS,0


In [43]:
allplayerFantasyGameLogs.loc[(allplayerFantasyGameLogs.fullName.isnull()) & (allplayerFantasyGameLogs.Player_ID==2403),'fullName'] = 'Nene Hilario'

In [44]:
allplayerFantasyGameLogs[allplayerFantasyGameLogs['Player_ID']==2403].head()

,GAME_DATE,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName,position1,Team,OpponentTeam,HomeGame
80,2015-10-28,22015,2403,0021500004,WAS @ ORL,W,12,0,6,0.00,...,1,-13,False,False,6.50,Nene Hilario,PF,WAS,ORL,0
687,2015-10-30,22015,2403,0021500027,WAS @ MIL,W,22,3,3,1.00,...,9,17,False,False,24.50,Nene Hilario,PF,WAS,MIL,0
794,2015-10-31,22015,2403,0021500034,WAS vs. NYK,L,18,3,4,0.75,...,8,2,False,False,14.00,Nene Hilario,PF,WAS,NYK,1
1315,2015-11-04,22015,2403,0021500061,WAS vs. SAS,W,20,3,5,0.60,...,6,-1,False,False,11.75,Nene Hilario,PF,WAS,SAS,1
1648,2015-11-06,22015,2403,0021500076,WAS @ BOS,L,15,1,4,0.25,...,5,-20,False,False,9.00,Nene Hilario,PF,WAS,BOS,0


In [45]:
allplayerFantasyGameLogs.Team.unique()

array([u'NOP', u'CLE', u'DET', u'CHI', u'ATL', u'GSW', u'ORL', u'TOR',
       u'MEM', u'CHA', u'DAL', u'PHI', u'PHX', u'DEN', u'HOU', u'MIL',
       u'LAC', u'MIA', u'IND', u'WAS', u'NYK', u'BOS', u'SAC', u'LAL',
       u'OKC', u'MIN', u'SAS', u'POR', u'UTA', u'BKN'], dtype=object)